# Data Loading Methods in PyTorch


The goal of this notebook is to explore PyTorch's tools for lazily loading datasets from disk instead of storing the datasets in memory. During supervised training over large parameter spaces, it'll be important to store many ground states for many parameter configurations before computation, as calculating them on the fly becomes time-consuming.

The goal is a tool living in the CPU that loads relevant data from disk (possibly without order, like random-access memory) to feed it to the model on the GPU. **As the model is training on that data on the GPU, the CPU should fetch the next batch of data.**


In [6]:
import torch
import numpy as np
import itertools
from Hamiltonian import Ising, XYZ
from optimizer_supervised import Optimizer
from model import TransformerModel

In [4]:
iham = Ising(3)
energy, psi = iham.calc_ground(param=torch.tensor([1.3]))
print(energy)
print(psi)

-4.657965112114926
tensor([-0.5694,  0.2421,  0.2421, -0.2421,  0.2421, -0.2421, -0.2421,  0.5694],
       dtype=torch.float64)


c:\Users\csmuser\Desktop\Spandan_Suthar_Research\tqs\Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [5]:
hham = XYZ(5)
energy, psi = hham.calc_ground(param=torch.tensor([0.5, 0, 0.2]))
print(energy)
print(psi)

-5.7620871035666354
tensor([-1.2634e-17-4.2907e-18j,  2.0638e-02-1.3801e-02j,
        -1.4873e-02-1.4024e-02j, -1.0300e-17+1.0365e-16j,
        -2.9830e-02+5.1339e-03j, -1.4745e-16-9.8012e-17j,
         1.2490e-16+5.0307e-17j,  1.4131e-01+1.3325e-01j,
        -3.5634e-03+1.7197e-02j,  2.1684e-16-9.3675e-17j,
        -2.2204e-16+1.8908e-16j,  5.4781e-02-2.6438e-01j,
         8.8471e-17-1.1449e-16j, -3.1728e-01+2.1217e-01j,
         2.8342e-01-4.8779e-02j,  1.0408e-17+1.8735e-16j,
         2.7628e-02+5.4945e-03j, -8.0448e-17+8.5869e-17j,
         5.0524e-17-1.5005e-16j, -1.9609e-01+1.3113e-01j,
         4.5970e-17+9.5193e-17j,  4.5859e-01-7.8925e-02j,
        -4.2473e-01-8.4469e-02j, -1.1449e-16-7.2858e-17j,
        -1.0582e-16+4.5103e-17j, -2.6250e-01-5.2205e-02j,
         2.2864e-01+2.1560e-01j,  1.6653e-16-1.5266e-16j,
         3.3856e-02-1.6339e-01j, -1.3184e-16+1.8041e-16j,
         1.4658e-16-1.8041e-16j, -4.0742e-16-3.0910e-16j],
       dtype=torch.complex128)


The following functions are from optimization_tests.ipynb (or optimizer_supervised.py):


In [6]:
def generate_parameter_range(start, end, step):
    """
    A simple generator returning the next value in a range of values
    whenever called, according to a step size.
    """
    value = start
    while value < end:
        yield value
        value += step


def generate_parameter_points(parameter_ranges, step_sizes, distribution=None):
    """
    Generate all possible combinations of parameter values for a model
    (i.e., the Cartesian product of values of parameters in a slice of parameter space)

    Parameters:
        parameter_ranges: torch.Tensor of shape (2, n_parameters)
            The starting and ending values for each dimension of the slice of parameter space
        step_sizes: torch.Tensor of shape (n_parameters,)
            The step size for each dimension of the slice of parameter space
        distribution: N/A
            TODO: Not implemented

    """

    parameter_ranges = parameter_ranges.T  # TODO: remove

    if distribution is not None:
        raise NotImplementedError(
            "Sampling using a custom distribution is not implemented yet."
        )

    # Every possible individual parameter value for each parameter, in order
    parameter_ranges = [
        generate_parameter_range(start.item(), end.item(), step.item())
        for (start, end), step in zip(parameter_ranges, step_sizes)
    ]

    return itertools.product(*parameter_ranges)

In [7]:
param_ranges = torch.tensor([[0, -1, 0.2], [1, 1, 0.3]])
param_steps = torch.tensor([0.1, 0.1, 0.1])

print(param_ranges)
print(param_steps)

tensor([[ 0.0000, -1.0000,  0.2000],
        [ 1.0000,  1.0000,  0.3000]])
tensor([0.1000, 0.1000, 0.1000])


In [8]:
points_generator = generate_parameter_points(param_ranges, param_steps)

In [9]:
for point in points_generator:
    print(torch.tensor(point))

tensor([ 0.0000, -1.0000,  0.2000])
tensor([ 0.0000, -1.0000,  0.3000])
tensor([ 0.0000, -0.9000,  0.2000])
tensor([ 0.0000, -0.9000,  0.3000])
tensor([ 0.0000, -0.8000,  0.2000])
tensor([ 0.0000, -0.8000,  0.3000])
tensor([ 0.0000, -0.7000,  0.2000])
tensor([ 0.0000, -0.7000,  0.3000])
tensor([ 0.0000, -0.6000,  0.2000])
tensor([ 0.0000, -0.6000,  0.3000])
tensor([ 0.0000, -0.5000,  0.2000])
tensor([ 0.0000, -0.5000,  0.3000])
tensor([ 0.0000, -0.4000,  0.2000])
tensor([ 0.0000, -0.4000,  0.3000])
tensor([ 0.0000, -0.3000,  0.2000])
tensor([ 0.0000, -0.3000,  0.3000])
tensor([ 0.0000, -0.2000,  0.2000])
tensor([ 0.0000, -0.2000,  0.3000])
tensor([ 0.0000, -0.1000,  0.2000])
tensor([ 0.0000, -0.1000,  0.3000])
tensor([0.0000e+00, 1.4901e-08, 2.0000e-01])
tensor([0.0000e+00, 1.4901e-08, 3.0000e-01])
tensor([0.0000, 0.1000, 0.2000])
tensor([0.0000, 0.1000, 0.3000])
tensor([0.0000, 0.2000, 0.2000])
tensor([0.0000, 0.2000, 0.3000])
tensor([0.0000, 0.3000, 0.2000])
tensor([0.0000, 0.3000, 0

In [10]:
points_generator = generate_parameter_points(param_ranges, param_steps)

## Dataset Generation

In [11]:
import csv
import io

In [14]:
print(energy)

-5.7620871035666354


In [15]:
print(psi)

tensor([-1.2634e-17-4.2907e-18j,  2.0638e-02-1.3801e-02j,
        -1.4873e-02-1.4024e-02j, -1.0300e-17+1.0365e-16j,
        -2.9830e-02+5.1339e-03j, -1.4745e-16-9.8012e-17j,
         1.2490e-16+5.0307e-17j,  1.4131e-01+1.3325e-01j,
        -3.5634e-03+1.7197e-02j,  2.1684e-16-9.3675e-17j,
        -2.2204e-16+1.8908e-16j,  5.4781e-02-2.6438e-01j,
         8.8471e-17-1.1449e-16j, -3.1728e-01+2.1217e-01j,
         2.8342e-01-4.8779e-02j,  1.0408e-17+1.8735e-16j,
         2.7628e-02+5.4945e-03j, -8.0448e-17+8.5869e-17j,
         5.0524e-17-1.5005e-16j, -1.9609e-01+1.3113e-01j,
         4.5970e-17+9.5193e-17j,  4.5859e-01-7.8925e-02j,
        -4.2473e-01-8.4469e-02j, -1.1449e-16-7.2858e-17j,
        -1.0582e-16+4.5103e-17j, -2.6250e-01-5.2205e-02j,
         2.2864e-01+2.1560e-01j,  1.6653e-16-1.5266e-16j,
         3.3856e-02-1.6339e-01j, -1.3184e-16+1.8041e-16j,
         1.4658e-16-1.8041e-16j, -4.0742e-16-3.0910e-16j],
       dtype=torch.complex128)


In [16]:
# def memoize(ham, param_range, param_step, directory):
# 
#     points_generator = generate_parameter_points(param_range, param_step)
# 
#     with open(directory, "a") as f:
#         for point in points_generator:
#             point_tensor = torch.tensor(point)
#             energy, psi = ham.calc_ground(param=point_tensor)
#             writer = csv.writer(io.BufferedWriter(f))
#             row = 
#             writer.writerow([energy] + psi.tolist())
#         
# memoize(iham, param_ranges, param_steps, "test.csv")

In [18]:
import pandas as pd

In [19]:

def memoize(ham, param_range, param_step, directory, chunk_size):
    points_generator = generate_parameter_points(param_range, param_step)
    N = ham.n # TODO Hamiltonians should specify this on construction
    NUM_PARAMS, _ = len(param_range)
    CSV_COLUMNS = ["energy"] + [f"param_{i}" for i in range(NUM_PARAMS)] + [f"psi_{i}" for i in range(N)]

    # Initialize a Pandas DataFrame of chunk_size rows to store the ground state energy and wavefunction
    ground_df = pd.DataFrame(columns=CSV_COLUMNS)
    
    with open(directory, "a") as d:

        # Iterate over points in the parameter space slice, calculating the ground state 
        # energy via brute force eigendecomposition at each point
        for point in points_generator:
            point_tensor = torch.tensor(point)
            energy, psi = ham.calc_ground(param=point_tensor)
            row = [energy] + psi.tolist()
            ground_df.loc[len(ground_df)] = row

            if len(ground_df) >= chunk_size:
                ground_df.to_csv(d, index=False, header=False)
                ground_df = pd.DataFrame(columns=["energy"] + [f"psi_{i}" for i in range(N)])
     
        


SyntaxError: invalid syntax (2153502547.py, line 8)

In [ ]:
psi.tolist()

[(6.535028594656378e-17-3.29326409892472e-17j),
 (-0.023017877723499742+0.002903771622841847j),
 (-0.022104404116881554+0.020715625230426277j),
 (4.85722573273506e-17+2.393918396847994e-16j),
 (0.009356604678203831+0.009899188867766423j),
 (-1.249000902703301e-16+3.191891195797325e-16j),
 (1.249000902703301e-16-6.323067069935462e-16j),
 (0.21001894089410159-0.19682383871775952j),
 (0.027887103827307913-0.014597590049091922j),
 (2.7755575615628914e-17-1.0269562977782698e-15j),
 (3.608224830031759e-16+1.0547118733938987e-15j),
 (-0.42871700104345467+0.22441322939316016j),
 (-4.0245584642661925e-16-5.828670879282072e-16j),
 (0.3538608945953221-0.04464057184169812j),
 (-0.08889921639554305-0.09405443144840209j),
 (1.1212818867845087e-15+1.3877787807814457e-16j),
 (0.007878573334869646-0.018920995671942623j),
 (4.163336342344337e-16+5.828670879282072e-16j),
 (-4.163336342344337e-16-9.159339953157541e-16j),
 (0.21869806014935217-0.027589390675401865j),
 (8.014422459012849e-16+1.1102230246251

## GPU-Accelerated Ground State Label Calculation

CuPy provides a direct analogue to Scipy's sparse Hermitian eigensolver:

In [1]:
import cupy
from cupyx.scipy.sparse.linalg import eigsh as cuda_eigsh
from cupyx.scipy.sparse import csr_matrix as cuda_csr_matrix
from scipy.sparse.linalg import eigsh as eigsh 

In [4]:
import pickle

In [7]:
iham = Ising(25)

c:\Users\csmuser\Desktop\Spandan_Suthar_Research\tqs\Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [8]:
full_ham = iham.full_H(param = 1)
with open("full_ham.pkl", "wb") as f:
    pickle.dump(full_ham, f)
print(type(full_ham))

<class 'scipy.sparse._csr.csr_matrix'>


In [9]:
full_ham_cuda = cuda_csr_matrix(full_ham)
with open("full_ham_cuda.pkl", "wb") as f:
    pickle.dump(full_ham_cuda, f)
print(type(full_ham_cuda))

<class 'cupyx.scipy.sparse._csr.csr_matrix'>


In [11]:
import gc

In [12]:
del full_ham
del full_ham_cuda
gc.collect()

894

In [15]:
import time

In [16]:
full_ham_cuda = pickle.load(open("full_ham_cuda.pkl", "rb"))
start = time.time()
eigenvalues, eigenvectors = cuda_eigsh(full_ham_cuda, k=1, which="SA")
end = time.time()
del full_ham_cuda
gc.collect()
print(end - start, "s")

22.490004777908325 s


In [17]:
full_ham = pickle.load(open("full_ham.pkl", "rb"))
start = time.time()
eigenvalues, eigenvectors = eigsh(full_ham, k=1, which="SA")
end = time.time()
del full_ham
gc.collect()
print(end - start, "s")

In [ ]:
# NOTE: produced via Copilot:

import time
import numpy as np
import cupy as cp
from scipy.sparse.linalg import eigsh
from cupyx.scipy.sparse.linalg import eigsh as cuda_eigsh

def analyze_runtimes(chain_lengths, time_cutoff):
    eigsh_runtimes = []
    cuda_eigsh_runtimes = []

    for length in chain_lengths:
        # Create the Ising Hamiltonian
        iham = Ising(length)
        full_ham = iham.full_H(param=1)
        full_ham_cuda = cuda_csr_matrix(full_ham)

        # Run eigsh and measure the runtime
        start_time = time.time()
        eigsh(full_ham, k=1, which='SA')
        end_time = time.time()
        eigsh_runtime = end_time - start_time
        eigsh_runtimes.append(eigsh_runtime)

        # Run cuda_eigsh and measure the runtime
        start_time = time.time()
        cuda_eigsh(full_ham_cuda, k=1, which='SA')
        end_time = time.time()
        cuda_eigsh_runtime = end_time - start_time
        cuda_eigsh_runtimes.append(cuda_eigsh_runtime)

        # Check if the runtime exceeds the time cutoff
        if eigsh_runtime > time_cutoff or cuda_eigsh_runtime > time_cutoff:
            break

    return eigsh_runtimes, cuda_eigsh_runtimes